In [196]:
import numpy as np
import gensim as gn
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string
from scipy.sparse import csc_matrix, save_npz
import json
import os, re, math
import multiprocessing as mp
import pandas as pd

In [109]:
wc = load_dict("wc1.csv")

In [110]:
wnd = load_dict("wnd1.csv")

In [108]:
def load_dict(file):
    data = {}
    df = pd.read_csv(file)
    data = df.to_dict()
    return data

In [205]:
def sum_dict(d):
    s = 0
    for key in d:
        s = s + d[key][0]
    return s

def create_PPMI(wc, wnd):
    len_d = sum_dict(wc)
    len_wc = len(wc)
    PMI = csc_matrix((len_wc, len_wc),dtype=float) # scipy sparse matrix
    x_ind = 0
    y_ind = 0
    while(x_ind < len_wc and y_ind < len_wc):
        for key in wc:
            for key1 in wc:
                s = (key,key1)
                try: 
                    w = wnd[str(s)][0]
                    t = w * len_d
                    b = wc[key][0] * wc[key1][0]
                    fin = math.log(t/b)
                    PMI[x_ind, y_ind] = fin
                    #print("PMI[",x_ind,",",y_ind,"] = ",fin)
                    x_ind+=1
                except KeyError:
                    continue
            y_ind+=1
    return PMI

In [206]:
PMI = create_PPMI(wc,wnd)

/home/zanepage/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


IndexError: index (230460) out of range (>= 230460)